In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from google.colab import drive
import yfinance as yf
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Flatten, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from math import sqrt
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
drive.mount('/content/drive')
path = '/content/drive/MyDrive/Volatility/New_data_2000_2024/Not_normal_wd/'

Mounted at /content/drive


In [3]:
def set_index(df):
  df.index = pd.to_datetime(df['Date'])
  df.drop(columns=['Date'], inplace=True)

In [4]:
filename = 'CCV_data.csv'
HV_data = pd.read_csv(path + filename)

In [5]:
set_index(HV_data)

In [6]:
tm_data = HV_data

In [8]:
HV_data = HV_data.drop(columns=['CCV'])

In [9]:
HV_data['CCV'] = tm_data['CCV']

In [10]:
HV_data

,"('GARCH', 'normal', 0)","('GARCH', 'normal', 1)","('GARCH', 'gaussian', 0)","('GARCH', 'gaussian', 1)","('GARCH', 'ged', 0)","('GARCH', 'ged', 1)","('FIGARCH', 'normal', 0)","('FIGARCH', 'normal', 1)","('FIGARCH', 'gaussian', 0)","('FIGARCH', 'gaussian', 1)",...,unemployment_rate,usd_eur,usd_jpy,usd_gbp,usd_cny,usd_cad,usd_mxn,gt_data,log_returns,CCV
Date,,,,,,,,,,,,,,,,,,,,,
2000-02-02,0.039601,0.038301,0.039601,0.038301,0.040044,0.039199,0.038388,0.038388,0.038388,0.038388,...,11.596551,2.781970,307.252039,4.529620,23.415205,4.090860,26.750733,209.303607,-0.005604,0.038529
2000-02-03,0.037979,0.038355,0.037979,0.038355,0.041000,0.040608,0.038633,0.038633,0.038633,0.038633,...,11.596551,2.799436,311.491144,4.529266,23.412659,4.119569,26.561052,209.303607,-0.008693,0.039312
2000-02-04,0.036791,0.037755,0.036791,0.037755,0.038214,0.040159,0.036990,0.036990,0.036990,0.036990,...,11.419775,2.728548,306.262089,4.474254,23.415452,4.106629,26.433950,209.303607,0.000572,0.037268
2000-02-07,0.046593,0.055072,0.046593,0.055072,0.047968,0.057512,0.046474,0.046474,0.046474,0.046474,...,11.313708,2.733357,299.491542,4.460147,23.414391,4.140641,26.339020,209.303607,0.027307,0.043409
2000-02-08,0.049131,0.041984,0.049131,0.041984,0.049560,0.042487,0.046711,0.046711,0.046711,0.046711,...,11.313708,2.730104,301.761354,4.470789,23.415982,4.142338,26.135551,209.303607,0.010478,0.049799
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26,0.000120,-0.000665,0.000120,-0.000665,0.000456,-0.000424,0.000121,0.000121,0.000121,0.000121,...,0.000000,-0.002546,0.509117,-0.003041,0.005162,0.001980,-0.105925,0.404061,0.008246,0.000132
2024-08-27,-0.003889,-0.004798,-0.003889,-0.004798,-0.003539,-0.002953,-0.004368,-0.004368,-0.004368,-0.004368,...,0.000000,0.002051,-0.551543,-0.001202,-0.006788,-0.001556,-0.173948,0.404061,0.009336,0.000613
2024-08-28,0.000280,-0.001148,0.000280,-0.001148,0.000449,-0.000383,0.000483,0.000483,0.000483,0.000483,...,0.000000,0.000849,0.247487,0.000849,0.002263,0.002051,-0.219910,0.707107,0.010307,0.000140


# Split train, validation and test data

In [11]:
def split_data(data, target, train_end =datetime(2022, 5, 30), test_start=datetime(2022, 5, 31), test_size=0.1):
  test_data = data.loc[test_start:]

  train_val = data.loc[:train_end]
  train_data, val_data = train_test_split(train_val, test_size=test_size, shuffle=False)

  return train_data, val_data, test_data

In [12]:
def sliding_window_transform(X, y, timesteps):
  X_windows = []
  y_windows = []
  for i in range(len(X) - timesteps + 1):
    X_window = X[i:i + timesteps]
    y_window = y[i + timesteps - 1]
    X_windows.append(X_window)
    y_windows.append(y_window)

  return np.array(X_windows), np.array(y_windows)

In [13]:
# Define the rolling window function
def rolling_window(df, in_sample_window_size, out_of_sample_size):
  X, y = [], []
  for i in range(in_sample_window_size, len(df) - out_of_sample_size):
    X.append(df.iloc[i - in_sample_window_size:i, :-1].values)  # All features except the target column
    y.append(df.iloc[i:i + out_of_sample_size, -1].values)  # Target column

  return np.array(X), np.array(y)

# Model

In [14]:
def calculate_mape(actual, predicted):
  actual, predicted = np.array(actual), np.array(predicted)
  return np.mean(np.abs((actual - predicted) / actual)) * 100

## CCV

In [15]:
daily_train, daily_val, daily_test = split_data(HV_data, 'CCV')

In [16]:
in_sam_win_sz = 25
out_sam_win_sz = 5

In [17]:
HV_X_train, HV_y_train = rolling_window(daily_train, in_sam_win_sz, out_sam_win_sz)
HV_X_val, HV_y_val = rolling_window(daily_val, in_sam_win_sz, out_sam_win_sz)
HV_X_test, HV_y_test = rolling_window(daily_test, in_sam_win_sz, out_sam_win_sz)

In [18]:
# HV_X_train = HV_X_train.reshape(HV_X_train.shape[0], win_sz, -1)
# HV_X_val = HV_X_val.reshape(HV_X_val.shape[0], win_sz, -1)
# HV_X_test = HV_X_test.reshape(HV_X_test.shape[0], win_sz, -1)

### LSTM

In [19]:
lstm_garch_model = Sequential()
lstm_garch_model.add(LSTM(48, input_shape=(HV_X_train.shape[1], HV_X_train.shape[2]), return_sequences=True))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(LSTM(16))
lstm_garch_model.add(Dropout(0.1))
lstm_garch_model.add(Dense(5))

lstm_garch_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [20]:
lstm_garch_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 25, 48)              │          15,744 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 16)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,989 (78.08 KB)

 Trainable params: 19,989 (78.08 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
lstm_history = lstm_garch_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 33ms/step - loss: 0.0139 - val_loss: 9.0341e-06
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 1.8411e-04 - val_loss: 4.9880e-06
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 1.2086e-04 - val_loss: 3.3087e-06
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 30ms/step - loss: 1.1040e-04 - val_loss: 3.2199e-06
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - loss: 8.7245e-05 - val_loss: 4.7495e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 31ms/step - loss: 9.3587e-05 - val_loss: 1.9687e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - loss: 6.3937e-05 - val_loss: 1.7988e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 7.1140e-05 - val_loss: 1.8823e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 28ms/step - loss: 6.7853e-05 - val_loss: 1.5741e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 32ms/step - loss: 6.9979e-05 - val_loss: 3.5597e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 24ms/ste

In [22]:
# Predictions
HV_lstm_predictions = lstm_garch_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step


In [23]:
mse = mean_squared_error(HV_y_test, HV_lstm_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_lstm_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_lstm_predictions)
print(f'MAPE: {mape}%')

MSE: 1.981313807535636e-06
RMSE: 0.001407591491710445
MAE: 0.0011690879876635556
MAPE: 11675.706783958583%


### GRU

In [24]:
gru_model = Sequential()
gru_model.add(GRU(48, input_shape=(HV_X_train.shape[1], HV_X_train.shape[2]), return_sequences=True))
gru_model.add(Dropout(0.1))
gru_model.add(GRU(16))
gru_model.add(Dropout(0.1))
gru_model.add(Dense(5))

gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [25]:
gru_model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 25, 48)              │          11,952 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 16)                  │           3,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 15,205 (59.39 KB)

 Trainable params: 15,205 (59.39 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
gru_history = gru_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 44ms/step - loss: 0.0892 - val_loss: 3.9494e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - loss: 2.0435e-04 - val_loss: 8.1876e-06
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 30ms/step - loss: 1.3753e-04 - val_loss: 4.4246e-06
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 41ms/step - loss: 8.9880e-05 - val_loss: 2.9807e-06
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 29ms/step - loss: 9.2069e-05 - val_loss: 2.7315e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 34ms/step - loss: 1.0107e-04 - val_loss: 2.7633e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 30ms/step - loss: 8.5644e-05 - val_loss: 2.4262e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step - loss: 8.9879e-05 - val_loss: 2.2636e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step - loss: 7.9197e-05 - val_loss: 2.4190e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 42ms/step - loss: 7.2458e-05 - val_loss: 1.9036e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 31ms/st

In [27]:
# Predictions
HV_gru_predictions = gru_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step


In [28]:
mse = mean_squared_error(HV_y_test, HV_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 1.5650252823252884e-05
RMSE: 0.003956040043181171
MAE: 0.0037157592221555924
MAPE: 36377.196991913086%


### LSTM_GRU

In [29]:
lstm_gru_model = Sequential()
lstm_gru_model.add(LSTM(48, input_shape=(HV_X_train.shape[1], HV_X_train.shape[2]), return_sequences=True))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(GRU(16))
lstm_gru_model.add(Dropout(0.1))
lstm_gru_model.add(Dense(5))

lstm_gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [30]:
lstm_gru_model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                        │ (None, 25, 48)              │          15,744 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 25, 48)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_2 (GRU)                          │ (None, 16)                  │           3,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 18,997 (74.21 KB)

 Trainable params: 18,997 (74.21 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
lstm_gru_history = lstm_gru_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 41ms/step - loss: 0.0171 - val_loss: 2.4399e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 3.2078e-04 - val_loss: 3.2629e-05
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - loss: 1.7682e-04 - val_loss: 3.9629e-06
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - loss: 1.2313e-04 - val_loss: 2.0971e-05
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 26ms/step - loss: 1.0141e-04 - val_loss: 3.9283e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 40ms/step - loss: 9.7540e-05 - val_loss: 1.9277e-05
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 7.6769e-05 - val_loss: 9.8635e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 25ms/step - loss: 7.5258e-05 - val_loss: 2.1794e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 37ms/step - loss: 6.5678e-05 - val_loss: 1.2028e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - loss: 7.5470e-05 - val_loss: 7.0458e-07
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 38ms/ste

In [32]:
# Predictions
HV_lstm_gru_predictions = lstm_gru_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step


In [33]:
mse = mean_squared_error(HV_y_test, HV_lstm_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_lstm_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_lstm_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 7.692500232221643e-05
RMSE: 0.008770689957022562
MAE: 0.008653026187063066
MAPE: 95842.56285407873%


In [34]:
# path_1 = '/content/drive/MyDrive/Volatility/Predictions/GKV/'

# df_pred = pd.DataFrame(HV_lstm_gru_predictions[:,0], columns=['lstm_gru_pred'])
# filename = 'LSTM_GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)

### BiLSTM

In [35]:
bilstm_model = Sequential()
bilstm_model.add(Bidirectional(LSTM(48, return_sequences=True), input_shape=(HV_X_train.shape[1], HV_X_train.shape[2])))
bilstm_model.add(Dropout(0.1))
bilstm_model.add(Bidirectional(LSTM(16)))
bilstm_model.add(Dropout(0.1))
bilstm_model.add(Dense(5))

bilstm_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [36]:
bilstm_model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional (Bidirectional)        │ (None, 25, 96)              │          31,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ (None, 25, 96)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_1 (Bidirectional)      │ (None, 32)                  │          14,464 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ (None, 32)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 5)                   │             165 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 46,117 (180.14 KB)

 Trainable params: 46,117 (180.14 KB)

 Non-trainable params: 0 (0.00 B)

In [37]:
bilstm_history = bilstm_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 15s 45ms/step - loss: 0.0129 - val_loss: 1.4676e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 51ms/step - loss: 1.4473e-04 - val_loss: 4.9899e-06
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 7s 43ms/step - loss: 9.9396e-05 - val_loss: 2.5442e-06
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 10s 40ms/step - loss: 9.7577e-05 - val_loss: 5.1423e-06
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - loss: 8.6915e-05 - val_loss: 1.6165e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 55ms/step - loss: 8.6390e-05 - val_loss: 6.7181e-06
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - loss: 7.7597e-05 - val_loss: 2.4398e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 56ms/step - loss: 7.2766e-05 - val_loss: 3.8500e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - loss: 8.1579e-05 - val_loss: 5.2682e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 9s 55ms/step - loss: 6.2436e-05 - val_loss: 1.7094e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms

In [38]:
# Predictions
HV_bilstm_predictions = bilstm_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step


In [39]:
mse = mean_squared_error(HV_y_test, HV_bilstm_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_bilstm_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_bilstm_predictions)
print(f'MAPE: {mape}%')

MSE: 1.7843651030291774e-05
RMSE: 0.00422417459751509
MAE: 0.0033460824019105988
MAPE: 35183.68710648045%


### BiLSTM_GRU

In [40]:
bilstm_gru_model = Sequential()
bilstm_gru_model.add(Bidirectional(LSTM(48, return_sequences=True), input_shape=(HV_X_train.shape[1], HV_X_train.shape[2])))
bilstm_gru_model.add(Dropout(0.1))
bilstm_gru_model.add(GRU(16))
bilstm_gru_model.add(Dropout(0.1))
bilstm_gru_model.add(Dense(5))

bilstm_gru_model.compile(optimizer=Adam(learning_rate=0.01), loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [41]:
bilstm_gru_model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ bidirectional_2 (Bidirectional)      │ (None, 25, 96)              │          31,488 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_8 (Dropout)                  │ (None, 25, 96)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (None, 16)                  │           5,472 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 5)                   │              85 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 37,045 (144.71 KB)

 Trainable params: 37,045 (144.71 KB)

 Non-trainable params: 0 (0.00 B)

In [42]:
bilstm_gru_history = bilstm_gru_model.fit(HV_X_train, HV_y_train, epochs=80, batch_size=32, validation_data=(HV_X_val, HV_y_val))

Epoch 1/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 14s 54ms/step - loss: 0.0263 - val_loss: 5.4966e-05
Epoch 2/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - loss: 4.0683e-04 - val_loss: 5.3069e-05
Epoch 3/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 12s 45ms/step - loss: 1.9827e-04 - val_loss: 1.2242e-05
Epoch 4/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 48ms/step - loss: 1.4711e-04 - val_loss: 8.9403e-06
Epoch 5/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - loss: 1.0122e-04 - val_loss: 8.1192e-06
Epoch 6/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 12s 51ms/step - loss: 9.1022e-05 - val_loss: 1.2489e-05
Epoch 7/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 35ms/step - loss: 1.0384e-04 - val_loss: 4.7384e-06
Epoch 8/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 8s 50ms/step - loss: 9.6571e-05 - val_loss: 1.6458e-06
Epoch 9/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 35ms/step - loss: 7.6385e-05 - val_loss: 2.6635e-06
Epoch 10/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 11s 43ms/step - loss: 7.6997e-05 - val_loss: 1.1041e-06
Epoch 11/80
158/158 ━━━━━━━━━━━━━━━━━━━━ 6s 36ms

In [43]:
# Predictions
HV_bilstm_gru_predictions = bilstm_gru_model.predict(HV_X_test)

17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step


In [44]:
mse = mean_squared_error(HV_y_test, HV_bilstm_gru_predictions)
print(f'MSE: {mse}')

rmse = np.sqrt(mse)
print(f'RMSE: {rmse}')

mae = mean_absolute_error(HV_y_test, HV_bilstm_gru_predictions)
print(f'MAE: {mae}')

mape = calculate_mape(HV_y_test, HV_bilstm_gru_predictions)
print(f'MAPE: {mape}%')

MSE: 5.047211978356515e-06
RMSE: 0.0022466000931088102
MAE: 0.0016256147971412016
MAPE: 16645.354378091197%


In [45]:
# path_1 = '/content/drive/MyDrive/Volatility/New_data_2000_2024/New_predictions/CCV/'

# df_pred = pd.DataFrame(HV_lstm_predictions[:,0], columns=['lstm_pred'])
# filename = 'LSTM_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_gru_predictions[:,0], columns=['gru_pred'])
# filename = 'GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_lstm_gru_predictions[:,0], columns=['lstm_gru_pred'])
# filename = 'LSTM_GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_bilstm_predictions[:,0], columns=['bilstm_pred'])
# filename = 'BiLSTM_pred.csv'
# df_pred.to_csv(path_1 + filename)


# df_pred = pd.DataFrame(HV_bilstm_gru_predictions[:,0], columns=['bilstm_gru_pred'])
# filename = 'BiLSTM_GRU_pred.csv'
# df_pred.to_csv(path_1 + filename)